## CapStone Project - Coursera
### This notebook will be used for the Capstone Project going forward

In [1]:
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
Hello Capstone Project Course!


In [2]:
#!pip install bs4
import requests
import math
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
!pip install folium
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 4.8 MB/s  eta 0:00:01
     |██████████████████████████

In [4]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
html_data = response.content

In [5]:
bsoup = BeautifulSoup(html_data, "html5lib")

In [6]:
print(bsoup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b1b417c8-ac37-4606-8f53-7da4755174e4","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":1019189119,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b1b417c8-ac37-4606-8f53-7da4755174e4","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":1019189119,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia

In [7]:
table_contents=[]
table=bsoup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [8]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [10]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [11]:
df['latitude']=""

In [12]:
#get latitude and logitude values through "Neighborhood, Borough" format from our df

for i in range(len(df)) :  
    address = df.loc[i,"Neighborhood"].split(',')[0] + ', ' + df.loc[i,"Borough"]
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        df.loc[i, "latitude"] = latitude
        df.loc[i, "longitude"] = longitude
    except:
        pass
        
df.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7611,-79.324059
1,M4A,North York,Victoria Village,43.7327,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6607,-79.360457
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7221,-79.437507
4,M7A,Queen's Park,Ontario Provincial Government,,NaN


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7611,-79.324059
1,M4A,North York,Victoria Village,43.7327,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6607,-79.360457
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7221,-79.437507
4,M7A,Queen's Park,Ontario Provincial Government,,NaN


In [13]:
#for the NaN values, get latitude and logitude values through postal code from our df

for i in range(len(df)) :
    address = df.loc[i,"PostalCode"]
    try:
        if math.isnan(df.loc[i,"latitude"]) or math.isnan(df.loc[i,"longitude"]):
            geolocator = Nominatim(user_agent="toronto_explorer")
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            df.loc[i, "latitude"] = latitude
            df.loc[i, "longitude"] = longitude
    except:
        pass    

df["latitude"] = pd.to_numeric(df["latitude"], downcast="float")
df["longitude"] = pd.to_numeric(df["longitude"], downcast="float")
df.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311188
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722080,-79.437508
4,M7A,Queen's Park,Ontario Provincial Government,NaN,NaN


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311188
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722080,-79.437508
4,M7A,Queen's Park,Ontario Provincial Government,NaN,NaN


In [14]:
Nancount = 0
for i in range(len(df)) :
    if math.isnan(df.loc[i,"latitude"]) or math.isnan(df.loc[i,"longitude"]):
        Nancount = Nancount + 1
print("rows having NaN values: ",Nancount)
df

rows having NaN values:  6


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311188
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722080,-79.437508
4,M7A,Queen's Park,Ontario Provincial Government,NaN,NaN
5,M9A,Etobicoke,Islington Avenue,43.716415,-79.555847
6,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221703
7,M3B,North York,Don Mills North,43.775349,-79.345947
8,M4B,East York,"Parkview Hill, Woodbine Gardens",40.710583,-73.797874
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.656498,-79.377113


rows having NaN values:  6


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311188
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722080,-79.437508
4,M7A,Queen's Park,Ontario Provincial Government,NaN,NaN
5,M9A,Etobicoke,Islington Avenue,43.716415,-79.555847
6,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221703
7,M3B,North York,Don Mills North,43.775349,-79.345947
8,M4B,East York,"Parkview Hill, Woodbine Gardens",40.710583,-73.797874
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.656498,-79.377113


#### We've almost got Coordinates for all rows, lets handle the rest

In [15]:
#M7A - Queen's Park - Ontario Provincial Government : 43.6623, -79.3895
#M5E - Downtown Toronto - Berczy Park : 43.6456872, -79.3827793
#M6E - York - Caledonia-Fairbanks : 43.6874536, -79.4677807
#M7R - Mississauga - Enclave of L4W : 43.6368591, -79.6204199
#M5W - Downtown Toronto Stn A - Enclave of M5E : 43.6464418, -79.3770757
#M7Y - East Toronto Business - Enclave of M4L : 43.6627906, -79.3234816

df.loc[df.PostalCode=='M7A', 'latitude'] = 43.6623
df.loc[df.PostalCode=='M7A', 'longitude'] = -79.3895

df.loc[df.PostalCode=='M5E', 'latitude'] = 43.6456872
df.loc[df.PostalCode=='M5E', 'longitude'] = -79.3827793

df.loc[df.PostalCode=='M6E', 'latitude'] = 43.6874536
df.loc[df.PostalCode=='M6E', 'longitude'] = -79.4677807

df.loc[df.PostalCode=='M7R', 'latitude'] = 43.6368591
df.loc[df.PostalCode=='M7R', 'longitude'] = -79.6204199

df.loc[df.PostalCode=='M5W', 'latitude'] = 43.6464418
df.loc[df.PostalCode=='M5W', 'longitude'] = -79.3770757

df.loc[df.PostalCode=='M7Y', 'latitude'] = 43.6627906
df.loc[df.PostalCode=='M7Y', 'longitude'] = -79.3234816

#lets have a look
df

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311188
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722080,-79.437508
4,M7A,Queen's Park,Ontario Provincial Government,43.662300,-79.389503
5,M9A,Etobicoke,Islington Avenue,43.716415,-79.555847
6,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221703
7,M3B,North York,Don Mills North,43.775349,-79.345947
8,M4B,East York,"Parkview Hill, Woodbine Gardens",40.710583,-73.797874
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.656498,-79.377113


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311188
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.722080,-79.437508
4,M7A,Queen's Park,Ontario Provincial Government,43.662300,-79.389503
5,M9A,Etobicoke,Islington Avenue,43.716415,-79.555847
6,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221703
7,M3B,North York,Don Mills North,43.775349,-79.345947
8,M4B,East York,"Parkview Hill, Woodbine Gardens",40.710583,-73.797874
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.656498,-79.377113


In [17]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
dttoronto_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dttoronto_data.head()

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.656498,-79.377113
2,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
3,M5E,Downtown Toronto,Berczy Park,43.645687,-79.382782
4,M5G,Downtown Toronto,Central Bay Street,43.654175,-79.380814


,PostalCode,Borough,Neighborhood,latitude,longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.660706,-79.360458
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.656498,-79.377113
2,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
3,M5E,Downtown Toronto,Berczy Park,43.645687,-79.382782
4,M5G,Downtown Toronto,Central Bay Street,43.654175,-79.380814


In [28]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="dttoronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.
The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [29]:
# create map of Downtown Toronto using latitude and longitude values
map_dtToronto = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(dttoronto_data['latitude'], dttoronto_data['longitude'], dttoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dtToronto)  
    
map_dtToronto

In [26]:
CLIENT_ID = 'CUTGW3DPP1TPGDSELZT1JMQGR0SWNUOOFJJGGHDOKAZJF0OE' # your Foursquare ID
CLIENT_SECRET = 'KWUNSTR2RZ0CAS4ORCD3DTL2LATB5NHBF3I0R03ZDZXIGFWI' # your Foursquare Secret
VERSION = '20210611' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CUTGW3DPP1TPGDSELZT1JMQGR0SWNUOOFJJGGHDOKAZJF0OE
CLIENT_SECRET:KWUNSTR2RZ0CAS4ORCD3DTL2LATB5NHBF3I0R03ZDZXIGFWI
Your credentails:
CLIENT_ID: CUTGW3DPP1TPGDSELZT1JMQGR0SWNUOOFJJGGHDOKAZJF0OE
CLIENT_SECRET:KWUNSTR2RZ0CAS4ORCD3DTL2LATB5NHBF3I0R03ZDZXIGFWI


In [30]:
dttoronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

'Regent Park, Harbourfront'

In [32]:
neighborhood_latitude = dttoronto_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = dttoronto_data.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = dttoronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.66070556640625, -79.36045837402344.
Latitude and longitude values of Regent Park, Harbourfront are 43.66070556640625, -79.36045837402344.


In [33]:
# type your answer here

#The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CUTGW3DPP1TPGDSELZT1JMQGR0SWNUOOFJJGGHDOKAZJF0OE&client_secret=KWUNSTR2RZ0CAS4ORCD3DTL2LATB5NHBF3I0R03ZDZXIGFWI&v=20210611&ll=43.66070556640625,-79.36045837402344&radius=500&limit=100'

'https://api.foursquare.com/v2/venues/explore?&client_id=CUTGW3DPP1TPGDSELZT1JMQGR0SWNUOOFJJGGHDOKAZJF0OE&client_secret=KWUNSTR2RZ0CAS4ORCD3DTL2LATB5NHBF3I0R03ZDZXIGFWI&v=20210611&ll=43.66070556640625,-79.36045837402344&radius=500&limit=100'

In [34]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c3dd8babda580c63a707e7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Regent Park',
  'headerFullLocation': 'Regent Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.66520557090625,
    'lng': -79.35424970773282},
   'sw': {'lat': 43.65620556190625, 'lng': -79.36666704031406}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5092fe87e4b0476c6e7375e2',
       'name': 'Regent Park Aquatic Centre',
       'location': {'address': '640 Dundas St. East',
        'crossStreet': 'at Sumach St.',
        'lat': 43.6606003130796,
        'lng': -79.36139162915349,
        'lab

{'meta': {'code': 200, 'requestId': '60c3dd8babda580c63a707e7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Regent Park',
  'headerFullLocation': 'Regent Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.66520557090625,
    'lng': -79.35424970773282},
   'sw': {'lat': 43.65620556190625, 'lng': -79.36666704031406}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5092fe87e4b0476c6e7375e2',
       'name': 'Regent Park Aquatic Centre',
       'location': {'address': '640 Dundas St. East',
        'crossStreet': 'at Sumach St.',
        'lat': 43.6606003130796,
        'lng': -79.36139162915349,
        'lab

In [35]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Paintbox Bistro,Restaurant,43.660050,-79.362855
3,Qi sushi,Sushi Restaurant,43.662552,-79.364258
4,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Regent Park Aquatic Centre,Pool,43.660600,-79.361392
1,Sumach Espresso,Coffee Shop,43.658135,-79.359515
2,Paintbox Bistro,Restaurant,43.660050,-79.362855
3,Qi sushi,Sushi Restaurant,43.662552,-79.364258
4,Daniels Spectrum,Performing Arts Venue,43.660137,-79.361808


In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.
20 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Downtown Toronto
#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
dtTorontovenues = getNearbyVenues(names=dttoronto_data['Neighborhood'],
                                   latitudes=dttoronto_data['latitude'],
                                   longitudes=dttoronto_data['longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [42]:
print(dtTorontovenues.shape)
dtTorontovenues.head()

(1085, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.660706,-79.360458,Regent Park Aquatic Centre,43.660600,-79.361392,Pool
1,"Regent Park, Harbourfront",43.660706,-79.360458,Sumach Espresso,43.658135,-79.359515,Coffee Shop
2,"Regent Park, Harbourfront",43.660706,-79.360458,Paintbox Bistro,43.660050,-79.362855,Restaurant
3,"Regent Park, Harbourfront",43.660706,-79.360458,Qi sushi,43.662552,-79.364258,Sushi Restaurant
4,"Regent Park, Harbourfront",43.660706,-79.360458,Daniels Spectrum,43.660137,-79.361808,Performing Arts Venue


In [43]:
dtTorontovenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,78,78,78,78,78,78
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",54,54,54,54,54,54
Central Bay Street,84,84,84,84,84,84
Christie,57,57,57,57,57,57
Church and Wellesley,59,59,59,59,59,59
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",62,62,62,62,62,62
"Harbourfront East, Union Station, Toronto Islands",43,43,43,43,43,43


In [44]:
print('There are {} uniques categories.'.format(len(dtTorontovenues['Venue Category'].unique())))

There are 167 uniques categories.


## 3. Analyze Each Neighborhood


In [45]:
# one hot encoding
dtToronto_onehot = pd.get_dummies(dtTorontovenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtToronto_onehot['Neighborhood'] = dtTorontovenues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dtToronto_onehot.columns[-1]] + list(dtToronto_onehot.columns[:-1])
dtToronto_onehot = dtToronto_onehot[fixed_columns]

dtToronto_onehot.head()

,Yoga Studio,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hospital,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [47]:
dtToronto_onehot.shape

(1085, 167)

In [48]:
dtToronto_grouped = dtToronto_onehot.groupby('Neighborhood').mean().reset_index()
dtToronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hospital,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.025641,0.00,0.025641,0.012821,0.000000,0.025641,0.00,0.000000,0.00,0.025641,0.012821,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.012821,0.076923,0.000000,0.000000,0.000000,0.000000,0.012821,0.012821,0.012821,0.128205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.00,0.000000,0.038462,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.038462,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051282,0.012821,0.000000,0.000000,0.000000,0.000000,0.012821,0.025641,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.025641,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.012821,0.00,0.000000,0.000000,0.025641,0.000000,0.00,0.012821,0.000000,0.000000,0.012821,0.000000,0.000000,0.038462,0.000000,0.000000,0.025641,0.000000,0.038462,0.012821,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012821,0.025641,0.012821,0.012821,0.000000,0.012821,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,0.000000,0.000000,0.012821
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.018519,0.000000,0.00,0.037037,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.

In [50]:
dtToronto_grouped.shape

(17, 167)

In [51]:
num_top_venues = 5

for hood in dtToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dtToronto_grouped[dtToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
           venue  freq
0    Coffee Shop  0.13
1           Café  0.08
2          Hotel  0.05
3  Deli / Bodega  0.04
4     Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0             Hotel  0.09
1       Coffee Shop  0.07
2       Pizza Place  0.06
3  Baseball Stadium  0.06
4    Scenic Lookout  0.04


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.14
1    Clothing Store  0.07
2    Sandwich Place  0.06
3        Restaurant  0.04
4  Sushi Restaurant  0.04


----Christie----
               venue  freq
0  Korean Restaurant  0.26
1        Coffee Shop  0.05
2     Sandwich Place  0.04
3          Gift Shop  0.04
4    Bubble Tea Shop  0.04


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.08
2              Gay Bar  0.08
3   Falafel Restaurant  0.03
4          Pizza Place  0.0

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dtToronto_grouped['Neighborhood']

for ind in np.arange(dtToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Sandwich Place,Restaurant,Deli / Bodega,Gym,Pharmacy,Japanese Restaurant,Salad Place
1,"CN Tower, King and Spadina, Railway Lands, Har...",Hotel,Coffee Shop,Pizza Place,Baseball Stadium,Gym,Sandwich Place,Scenic Lookout,Concert Hall,Aquarium,Thai Restaurant
2,Central Bay Street,Coffee Shop,Clothing Store,Sandwich Place,Restaurant,Hotel,Middle Eastern Restaurant,Sushi Restaurant,Gym,Plaza,Pizza Place
3,Christie,Korean Restaurant,Coffee Shop,Indian Restaurant,Sandwich Place,Bubble Tea Shop,Pub,Fast Food Restaurant,Café,Gift Shop,Dessert Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Gastropub,Hotel,Pizza Place,Falafel Restaurant,Diner,Cosmetics Shop,Ramen Restaurant


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [55]:
# set number of clusters
kclusters = 5

dtToronto_grouped_clustering = dtToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtToronto_merged = dttoronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
dtToronto_merged = dtToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dtToronto_merged.tail() # check the last columns!

,PostalCode,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.642563,-79.387085,0,Hotel,Coffee Shop,Pizza Place,Baseball Stadium,Gym,Sandwich Place,Scenic Lookout,Concert Hall,Aquarium,Thai Restaurant
13,M4W,Downtown Toronto,Rosedale,43.663288,-79.419014,1,Korean Restaurant,Café,Pub,Coffee Shop,Gift Shop,Grocery Store,Bubble Tea Shop,Indian Restaurant,Fast Food Restaurant,Japanese Restaurant
14,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.669403,-79.372704,3,Coffee Shop,Pizza Place,Hotel,Café,Grocery Store,Italian Restaurant,Diner,Pet Store,Pharmacy,Caribbean Restaurant
15,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648769,-79.381691,0,Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Gym,Asian Restaurant,Bank,Japanese Restaurant,Deli / Bodega
16,M4Y,Downtown Toronto,Church and Wellesley,43.662285,-79.376221,0,Coffee Shop,Japanese Restaurant,Gay Bar,Gastropub,Hotel,Pizza Place,Falafel Restaurant,Diner,Cosmetics Shop,Ramen Restaurant


Finally, let's visualize the resulting clusters


In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtToronto_merged['latitude'], dtToronto_merged['longitude'], dtToronto_merged['Neighborhood'], dtToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


#### Cluster 1


In [62]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 0, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Hotel,Sandwich Place,Café,Cosmetics Shop,Bookstore,Burger Joint,Pizza Place,Restaurant
3,Downtown Toronto,0,Coffee Shop,Café,Hotel,Sandwich Place,Restaurant,Deli / Bodega,Gym,Pharmacy,Japanese Restaurant,Salad Place
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Sandwich Place,Restaurant,Hotel,Middle Eastern Restaurant,Sushi Restaurant,Gym,Plaza,Pizza Place
6,Downtown Toronto,0,Coffee Shop,Café,Hotel,Gym,Clothing Store,Restaurant,Asian Restaurant,Sandwich Place,Bank,Cosmetics Shop
7,Downtown Toronto,0,Coffee Shop,Hotel,Scenic Lookout,Baseball Stadium,Aquarium,Concert Hall,Pizza Place,Lounge,Cocktail Bar,Park
8,Downtown Toronto,0,Coffee Shop,Café,Hotel,Sandwich Place,Asian Restaurant,Pharmacy,Japanese Restaurant,American Restaurant,Restaurant,Deli / Bodega
9,Downtown Toronto,0,Coffee Shop,Sandwich Place,Hotel,Café,Restaurant,Gym,Japanese Restaurant,Cocktail Bar,Bank,Italian Restaurant
11,Downtown Toronto,0,Café,Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Pizza Place,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Nightclub,Light Rail Station
12,Downtown Toronto,0,Hotel,Coffee Shop,Pizza Place,Baseball Stadium,Gym,Sandwich Place,Scenic Lookout,Concert Hall,Aquarium,Thai Restaurant
15,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Hotel,Restaurant,Gym,Asian Restaurant,Bank,Japanese Restaurant,Deli / Bodega


#### Cluster 2

In [63]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 1, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,1,Korean Restaurant,Coffee Shop,Indian Restaurant,Sandwich Place,Bubble Tea Shop,Pub,Fast Food Restaurant,Café,Gift Shop,Dessert Shop
13,Downtown Toronto,1,Korean Restaurant,Café,Pub,Coffee Shop,Gift Shop,Grocery Store,Bubble Tea Shop,Indian Restaurant,Fast Food Restaurant,Japanese Restaurant


#### Cluster 2

In [64]:
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 2, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,2,Café,Coffee Shop,Park,Bar,Japanese Restaurant,Sandwich Place,Italian Restaurant,Bank,Restaurant,Bubble Tea Shop
